<a href="https://colab.research.google.com/github/Sameer-30/Neural-Network-From-Scratch/blob/main/Calculating_Network_Error_with_Loss0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Calculating Categorical Cross-Entropy Loss & Accuracy

In this notebook, we demonstrate how to compute the categorical cross-entropy loss and accuracy for a neural network. Unlike previous examples, we use synthetic data to illustrate the steps in detail.

We'll cover:
- Generating synthetic softmax predictions for a batch of samples.
- Calculating the loss using sparse (index-based) targets.
- Converting targets to one-hot encoded format and calculating the loss.
- Computing the accuracy metric.



In [ ]:
import numpy as np

# Generate synthetic softmax predictions for a batch of 5 samples with 4 classes.
# These predictions might come from a softmax layer in a neural network.
predictions = np.array([
    [0.85, 0.05, 0.05, 0.05],
    [0.10, 0.70, 0.10, 0.10],
    [0.20, 0.20, 0.50, 0.10],
    [0.30, 0.30, 0.20, 0.20],
    [0.05, 0.15, 0.70, 0.10]
])

# Assume the true class for each sample is given as an index (sparse targets).
# For example, sample 0's true label is class 0, sample 1's is class 1, etc.
target_labels = np.array([0, 1, 2, 0, 2])

print("Synthetic Predictions:\n", predictions)
print("Target Labels (sparse):", target_labels)


## 1. Calculating Categorical Cross-Entropy Loss (Sparse Targets)
In many classification tasks, the neural network outputs a probability distribution over classes using a softmax activation. For each sample, the categorical cross-entropy loss is defined as:
$$
L = -\log(\hat{y}_k)
$$
where $\hat{y}_k$ is the predicted probability for the true class $(k)$.


When we have a batch of samples, our predicted probabilities are stored in a two-dimensional array of shape
(
batch_size
,
num_classes
)
(batch_size,num_classes). The true labels are given as a one-dimensional (sparse) array where each element is the index of the correct class.

Consider the following function that implements this loss calculation:

In [ ]:
def categorical_crossentropy_loss(y_pred, y_true):

    # Clip predictions to avoid log(0) issues
    y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

    # Extract the predicted probabilities for the correct classes
    # np.arange(len(y_pred)) creates an array of sample indices
    correct_confidences = y_pred_clipped[np.arange(len(y_pred)), y_true]

    # Compute the negative log likelihood for each sample
    negative_log_likelihoods = -np.log(correct_confidences)

    # Return the mean loss over the batch
    return np.mean(negative_log_likelihoods)

loss_sparse = categorical_crossentropy_loss(predictions, target_labels)
print("Categorical Cross-Entropy Loss (sparse targets):", loss_sparse)


## 2. Handling One-Hot Encoded Targets

Sometimes targets are provided as one-hot encoded vectors. For example, if the target label for a sample is class 2 among 4 classes, its one-hot representation is $[0, 0, 1, 0]$.

To calculate the loss, we multiply the predicted probabilities by the one-hot encoded target and sum along the class dimension.


In [ ]:
def to_one_hot(labels, num_classes):

    one_hot = np.zeros((labels.shape[0], num_classes))
    one_hot[np.arange(labels.shape[0]), labels] = 1
    return one_hot

# Convert our sparse target_labels to one-hot encoded format.
target_one_hot = to_one_hot(target_labels, 4)
print("One-Hot Encoded Targets:\n", target_one_hot)


In [ ]:
def categorical_crossentropy_loss_onehot(y_pred, y_true):
       # Clip predictions for numerical stability
    y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

    # For one-hot targets, multiply element-wise and sum along classes to get correct confidences
    correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

    # Compute the negative log likelihood
    negative_log_likelihoods = -np.log(correct_confidences)

    return np.mean(negative_log_likelihoods)

loss_onehot = categorical_crossentropy_loss_onehot(predictions, target_one_hot)
print("Categorical Cross-Entropy Loss (one-hot targets):", loss_onehot)


## 3. Calculating Accuracy

Accuracy is determined by comparing the predicted class (the index with the highest probability) with the true class.

For a batch of predictions, we compute:
$$
\text{Accuracy} = \frac{\text{Number of correct predictions}}{\text{Total number of samples}}
$$


In [ ]:
def calculate_accuracy(y_pred, y_true):
        # Determine the predicted class for each sample
    pred_labels = np.argmax(y_pred, axis=1)

    # If y_true is one-hot encoded, convert to sparse format
    if len(y_true.shape) == 2:
        y_true = np.argmax(y_true, axis=1)

    accuracy = np.mean(pred_labels == y_true)
    return accuracy

accuracy_val = calculate_accuracy(predictions, target_labels)
print("Accuracy (using sparse targets):", accuracy_val)
